<a href="https://colab.research.google.com/github/ErenB02/comp_bio/blob/Labs(DM)/LAB6(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining - Lab GridSearch


Lecturer: Dr Andrea Visentin


## Specification

The objective of this project is to build a k-Nearest Neighbour algorithm that takes as input training and test dataset and will predict the target variable with a reasonable degree of accuracy.

This project focuses on a modified version of the Real estate valuation data set (https://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set).

You can find the correct csv that you should use for the assignment here:
https://github.com/andvise/DataAnalyticsDatasets/blob/main/real_estate.csv


# Data Preparation


1.  Load the dataset on Colab


In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/andvise/DataAnalyticsDatasets/refs/heads/main/real_estate.csv")
df

,num,transaction_date,house_age,distance_MRT_station,number_convenience_stores,latitude,longitude,house_price_per_unit
0,1,2012.916667,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.916667,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583333,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500000,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833333,5.0,390.56840,5,24.97937,121.54245,43.1
...,...,...,...,...,...,...,...,...
409,410,2013.000000,13.7,4082.01500,0,24.94155,121.50381,15.4
410,411,2012.666667,5.6,90.45606,9,24.97433,121.54310,50.0
411,412,2013.250000,18.8,390.96960,7,24.97923,121.53986,40.6
412,413,2013.000000,8.1,104.81010,5,24.96674,121.54067,52.5




2. Delete the columns **num** and **transaction_date**



In [2]:
del df['num']
del df['transaction_date']

3. Use **house_price_per_unit** as the target value and the rest of the data frame as features


In [3]:
y = df['house_price_per_unit']

#loc syntax = [row selection, column selection]
#We use : in row to get all rows
#We use : again but stop at longitude
X = df.loc[:, :"longitude"]

In [ ]:
X

,house_age,distance_MRT_station,number_convenience_stores,latitude,longitude
0,32.0,84.87882,10,24.98298,121.54024
1,19.5,306.59470,9,24.98034,121.53951
2,13.3,561.98450,5,24.98746,121.54391
3,13.3,561.98450,5,24.98746,121.54391
4,5.0,390.56840,5,24.97937,121.54245
...,...,...,...,...,...
409,13.7,4082.01500,0,24.94155,121.50381
410,5.6,90.45606,9,24.97433,121.54310
411,18.8,390.96960,7,24.97923,121.53986
412,8.1,104.81010,5,24.96674,121.54067


4. Using sklearn, divide your dataset into 80% for training and 20% for test


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

5. Scale the columns using min-max scalers using a MinMaxScaler (from sklearn import preprocessing)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(X_train)
## Equivalent to
# min_train = X_train.min()
# max_train = X_train.max()

X_train = scaler.transform(X_train)
## Equivalent to
# X_train = (X_train - min_train)/ (max_train - min_train)

X_test = scaler.transform(X_test)
## Equivalent to
# X_test = (X_test - min_train)/ (max_train - min_train)

#Stored original data
print(scaler.data_min_)
print(scaler.data_max_)

#Fit the scaler on the training data to learn the scaling parameters.
#Transform both training and test data using the same scaling parameters (from the training set).
#This ensures that the model is trained and evaluated on properly scaled data without any leakage from the test set


[  0.       23.38284   0.       24.93207 121.47516]
[  43.8     6306.153     10.        25.01459  121.56627]
[  0.       23.38284   0.       24.93207 121.47516]
[  43.8     6306.153     10.        25.01459  121.56627]
[  0.       23.38284   0.       24.93207 121.47516]
[  43.8     6306.153     10.        25.01459  121.56627]


In [ ]:
X_test.max()

1.0289471038042872

In [ ]:
scaler.data_min_

array([  0.     ,  23.38284,   0.     ,  24.93207, 121.47516])

In [ ]:
scaler.data_max_

array([  43.8    , 6306.153  ,   10.     ,   25.01459,  121.56627])

6. Print the shape of the train and test set  

In [ ]:
print(X_train.shape)
print(X_test.shape)

(331, 5)
(83, 5)


# k-NN Hyperparametrisation

1. Create a kNN regressor, https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
\

2. Define a dictionary of hyperparameters that include:
*   Manhattan and Euclidian distance
*   k = [1,3,5,7,9]
* Normal and weighted version of the kNN



In [ ]:
from sklearn import neighbors
from sklearn import metrics
from sklearn import model_selection
import matplotlib.pyplot as plt
#VERY IMPORTANT POINT HERE
#kNN Regressor is used for regression tasks, where the goal is to predict a continuous output value (e.g., predicting house prices, sales, etc.)
knn = neighbors.KNeighborsRegressor()
parameters = {'n_neighbors': [1, 3, 5, 7, 9], 'p': [1, 2]}

# Initialize the GridSearchCV object
# GridSearchCV will try all combinations of the given hyperparameters
# 'scoring' defines how to evaluate the model, we use 'neg_mean_squared_error' to minimize the MSE
clf = model_selection.GridSearchCV(knn, parameters, scoring='neg_mean_squared_error')
clf.fit(X_train, y_train)
# Use case: MAE measures the average magnitude of the errors in a set of predictions, without considering their direction (positive or negative).
#It is easier to interpret than MSE since it gives the average absolute difference between predicted and actual values.
# When to use: MAE is better when you care more about the absolute error, and you don't want to penalize large errors as severely as MSE.
#It's useful if you need a more robust metric against outliers, as large errors don't get squared.

# After fitting, you can access the best model and the corresponding parameters
clf.best_estimator_.score(X_test, y_test)


0.8048781739762458

In [ ]:
#Using negative MSE
#Thus we look for the best maximizing score
# -75 since we have negative MSE
clf.best_score_

-75.1287168199407

3. Find the best set of hyperparameters using Cross Validation, https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html



4. Print the best parameters

In [ ]:
#Retrieves the best parameters from GridSearchCV
clf.best_params_

{'n_neighbors': 3, 'p': 1}



```
# This is formatted as code
```

5. Evaluate the best model on the test set


In [ ]:
#clf.best_estimator_: The best model found after GridSearchCV.
#.score(X_test, y_test): Evaluates the best model on the test set
clf.best_estimator_.score(X_test, y_test)


0.8048781739762458